In [1]:
from newsapi import NewsApiClient


import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from dotenv import load_dotenv
import pandas as pd
import os

import datetime as dt
from dateutil.relativedelta import relativedelta

import plotly.express as px
import plotly.graph_objects as go

# The correct way to import newsapi-python is:


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\karahan.cetinkaya\AppData\Roaming\nltk_data..
[nltk_data]     .
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
query = 'crude oil'

# Step 1: Get today's date as string
today_str = dt.datetime.now().strftime('%Y-%m-%d')

# Step 2: Convert string to datetime object
today_dt = dt.datetime.strptime(today_str, '%Y-%m-%d')

# Step 3: Subtract one month
previous_day = today_dt - relativedelta(days=30)

# Step 4: Convert back to string if needed
previous_day_str = previous_day.strftime('%Y-%m-%d')

In [3]:
load_dotenv()

api_key = os.getenv('API_KEY')

newsapi = NewsApiClient(api_key)

all_articles = newsapi.get_everything(q=query,
                                       from_param=previous_day_str,
                                       to=today_str,
                                       sources='yfinance,google-news',
                                       domains='finance.yahoo.com,news.google.com,yahoo.com',
                                       exclude_domains='news.google.com',
                                       language='en',
                                       sort_by='relevancy',
                                       page_size=100)
articles = all_articles['articles']
df = pd.DataFrame(articles)

NewsAPIException: {'status': 'error', 'code': 'parameterInvalid', 'message': 'You are trying to request results too far in the past. Your plan permits you to request articles as far back as 2025-08-23, but you have requested 2025-03-08. You may need to upgrade to a paid plan.'}

In [ ]:
print(df)

                                         source  \
0   {'id': None, 'name': 'Yahoo Entertainment'}   
1   {'id': None, 'name': 'Yahoo Entertainment'}   
2   {'id': None, 'name': 'Yahoo Entertainment'}   
3   {'id': None, 'name': 'Yahoo Entertainment'}   
4   {'id': None, 'name': 'Yahoo Entertainment'}   
..                                          ...   
95  {'id': None, 'name': 'Yahoo Entertainment'}   
96  {'id': None, 'name': 'Yahoo Entertainment'}   
97  {'id': None, 'name': 'Yahoo Entertainment'}   
98  {'id': None, 'name': 'Yahoo Entertainment'}   
99  {'id': None, 'name': 'Yahoo Entertainment'}   

                              author  \
0                            Reuters   
1                      Brijesh Patel   
2   Hritam Mukherjee and Nidhi Verma   
3                  Nilanjan Banerjee   
4                            Reuters   
..                               ...   
95                      Florence Tan   
96                           Reuters   
97        RAJESH ROY and HU

In [ ]:
print(df.columns)

Index(['source', 'author', 'title', 'description', 'url', 'urlToImage',
       'publishedAt', 'content'],
      dtype='object')


In [ ]:
sia = SentimentIntensityAnalyzer()

analyze_data = []

analyze_data.append(['source','author','date', 'title', 'description','content','sentiment'])
for i in range(len(df)):
    url = df['url'][i]
    author = df['author'][i]
    title = df['title'][i]
    description = df['description'][i]
    date = df['publishedAt'][i]
    content = df['content'][i]
    if content is None:
        content = ''
    
    text = title + ' ' + content
    sentiment = sia.polarity_scores(text)
    
    analyze_data.append([url, author, date, title, description, content, sentiment['compound']])

df_sentiment = pd.DataFrame(analyze_data[1:], columns=analyze_data[0])

# Convert 'date' to datetime and 'sentiment' to float
df_sentiment['date'] = pd.to_datetime(df_sentiment['date'])
df_sentiment['sentiment'] = df_sentiment['sentiment'].astype(float)


# Group by date (remove time) and calculate average sentiment per day
df_sentiment['date_only'] = df_sentiment['date'].dt.date
daily_sentiment = df_sentiment.groupby('date_only')['sentiment'].mean().reset_index()



In [ ]:
import pandas as pd
import os

excel_path = "crude_oil_sentiment_score_.xlsx"  # Your Excel file path

# Convert analyze_data to DataFrame (skip header row for data)
columns = ['source', 'author', 'date', 'title', 'description', 'content', 'sentiment']
new_df = pd.DataFrame(analyze_data[1:], columns=analyze_data[0])

if os.path.exists(excel_path):
    # Read existing data
    old_df = pd.read_excel(excel_path)
    # Concatenate and drop duplicates (optional, based on 'date' and 'title')
    combined_df = pd.concat([old_df, new_df], ignore_index=True)
    combined_df = combined_df.drop_duplicates(subset=['date', 'title'])
else:
    combined_df = new_df

# Save back to Excel
combined_df.to_excel(excel_path, index=False)